In [ ]:
# Read train data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2

def getTrainObjects(objectsplit=1, upsamplingratio=1, positive_multiplier=1, porositythreshold=0.5, separate_test=True):
    emptyRatio = 47
    objectwidth = 83
    objectheight = 122
    xspacing = 133
    yspacing = 270
    xstart = 293
    ystart = 268
    xend = 1730
    yend = 1770
    hsegments = [0,26,50,74,98,122]
    powderthickness = 80
    endlayer = 187

    paths = pathlib.Path('./OT data 80 um/int').glob('*.tif')
    paths_sorted = [x for x in paths]
    paths_sorted.sort()
    block = np.array([np.array(plt.imread(path)) for path in paths_sorted])
    integrals = block[0:endlayer]

    del paths_sorted
    objectinfo = pd.read_csv('Parameters.csv', names=["Object", "P", "S", "H", "Porosity", "Label"])

    objectCoordinates = [[x, x+objectwidth, y, y+objectheight] for y in reversed(range(
        ystart, yend, objectheight + yspacing)) for x in range(xstart, xend, xspacing + objectwidth)]
    coorddf = pd.DataFrame(objectCoordinates, columns=['xstart', 'xend', 'ystart', 'yend'])
    objectinfo = coorddf.join(objectinfo)

    objects = np.full((len(objectinfo), endlayer, objectheight, objectwidth), np.nan)

    for index, object in objectinfo.iterrows():
        objects[index] = integrals[:, object.ystart:object.yend, object.xstart:object.xend]

    aggregate = np.sum(objects, axis=(0,1))

    emptyRatio = 47
    limit = np.percentile(aggregate, emptyRatio)
    mask = aggregate >= limit
    mask = np.repeat([mask], endlayer, 0)

    for object in objects:
        object[~mask] = np.nan

    # Time to construct the "real" dataframe

    segmentdf = pd.read_csv('Segments.csv', names=["Object", "Objectnumber", "Segment", "P", "S", "H", "Porosity", "Area"])
    segmentdf.insert(1, "VED", segmentdf.P * 1000/(segmentdf.S * segmentdf.H * powderthickness))
    segmentdf.insert(1, "Label", np.where(segmentdf.Porosity > porositythreshold, 1, 0))
    originalframe = segmentdf.copy()
    hs = [[hsegments[j], hsegments[j+1]] for i in range(0, len(objects)) for j in range(0, len(hsegments)-1)]
    coorddf = pd.DataFrame(hs, columns=['hstart', 'hend'])
    segmentdf = coorddf.join(segmentdf)

    # Start of object multiplication 
    layersPerObject = endlayer // objectsplit
    testEnd = endlayer if separate_test else endlayer - layersPerObject * (objectsplit // 3)
    zs = [segmentdf.copy().assign(zstart=z, zend=z+layersPerObject) for z in range(0, testEnd-layersPerObject+1, layersPerObject//(upsamplingratio * positive_multiplier))]
    testzs = [segmentdf.copy().assign(zstart=testEnd, zend=endlayer)]
    trainobjectinfo = pd.concat(zs, ignore_index=True)
    trainobjectinfo.drop(trainobjectinfo[(trainobjectinfo.Objectnumber == 28) | (trainobjectinfo.Objectnumber == 21) ].index, inplace=True)
    trainobjectinfo.reset_index(drop=True, inplace=True)
    testobjectinfo = pd.concat(testzs, ignore_index=True)
    testobjectinfo.drop(testobjectinfo[(testobjectinfo.Objectnumber == 28) | (testobjectinfo.Objectnumber == 21) ].index, inplace=True)
    testobjectinfo.reset_index(drop=True, inplace=True)

    # Removes extra rows 
    trainobjectinfo = trainobjectinfo[(trainobjectinfo['Label'] == 1) | (trainobjectinfo['zstart'] % (positive_multiplier) == 0)]

    trainobjects = [objects[object.Objectnumber-1, object.zstart:object.zend, object.hstart:object.hend] for index, object in trainobjectinfo.iterrows()]
    testobjects = [objects[object.Objectnumber-1, object.zstart:object.zend, object.hstart:object.hend] for index, object in testobjectinfo.iterrows()]

    print("fetching data with objectsplit: {}, upsamplingratio: {}, positive_multiplier: {}, porositythreshold: {}".format(objectsplit, upsamplingratio, positive_multiplier, porositythreshold))
    return trainobjects, trainobjectinfo, testobjects, testobjectinfo

# assert(np.average(np.isfinite(trainobjects)) == 1)
# assert(np.average(np.isfinite(testobjects)) == 1)
# xs = np.copy(aggregate)
# xs[~backgroundmask] = np.nan
# plt.imshow(xs)
# plt.figure()

In [ ]:
from sklearn import neighbors, metrics
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from datetime import datetime
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import tree
import warnings

def preprocess(objects, type, sharpening):
    rtn = []
    # print(rtn.shape)
    for index, object in enumerate(objects):
        object = np.copy(object)
        sharpened = object
        if(sharpening != 'none'):
            sharpeningKernel = np.array([   [-1, -1,  -1],
                                        [-1,  9,  -1],
                                        [ -1, -1,  -1]
        ]) if sharpening == 'diagonal' else np.array([  [0, -1,  0],
                                                        [-1, 5, -1],
                                                        [0, -1,  0]])
            sharpened = np.array([cv2.filter2D(src=image, ddepth=-1, kernel=sharpeningKernel) for image in object])
        # Sharpening is done
        if type == 'scatter' or type == 'spatstat':
            xs = np.array(sharpened, copy=True, dtype=np.float32)
            rtn.append(xs)
        elif type == 'moran':
            xs = np.array(sharpened, copy=True, dtype=np.float32)
            avg = np.nanmean(xs)
            stddev = np.nanstd(xs)
            xs = (xs - avg) / avg
            rtn.append(xs)
    return rtn


def calculateoutliers(objects, type, neighbourhoodSetting, windowSize):
    # c, z, y, x = objects.shape


    outlierValues = []
    index = 0
    # return objects
    for object in objects:
        object = np.copy(object)
        z, y, x = object.shape
        # Step 1: calculate neighbourhood
        neighbourkernel = np.ones((neighbourhoodSetting, neighbourhoodSetting)) / neighbourhoodSetting**2
        flatNeighbourhood = np.array([cv2.filter2D(src=layer, ddepth=-1, kernel=neighbourkernel) for layer in object])
        neighbourhoodValues = np.array([
            np.sum(flatNeighbourhood[layerIndex-windowSize:layerIndex], axis=0)/windowSize
            for layerIndex in range(windowSize, z+1)
        ])
        # Step 2: calculate outlier
        offset = windowSize // 2
        endoffset = windowSize - offset - 1

        ys = neighbourhoodValues[0:z-windowSize+1]
        xs = object[offset:z-endoffset]
        filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))

        # plt.imshow(xs[0])
        # plt.figure()
        # plt.imshow(xs[0])
        # plt.figure()
        # if(index == 58):
        #     plt.imshow(xs[0])
        #     plt.figure()
        #     plt.imshow(ys[0])
        #     plt.figure()
        #     plt.imshow(filter[0])
        #     plt.figure()
        #     print(len(np.unique(filter)))
        numberOfFilterValues = len(np.unique(filter))
        # print("filterlength is: ", numberOfFilterValues)
        # print("index is:", index)
        assert numberOfFilterValues == 2, f"Expected filter to have two values, got: {numberOfFilterValues}"
        if type == 'spatstat':
            outliers = xs - ys
            avg = np.mean(outliers[filter])
            std = np.std(outliers[filter])
            outliers = (outliers - avg) / std
            outlierValues.append(outliers)
        else:
            with warnings.catch_warnings():
                line = np.polyfit(xs[filter].flatten(), ys[filter].flatten(), 1)
                p = np.poly1d(line)
                outlierValues.append(p(xs) - ys)
            # plt.scatter(xs[filter].flatten(), ys[filter].flatten())
            # plt.axline((-0.1, p(-0.1)), (0, p(0)), linewidth=2, color='b')
            # plt.figure()
            assert(xs.shape == p(ys).shape)
        assert(len(np.unique(outlierValues[index])) > 1)
        assert(len(np.unique(np.isfinite(outlierValues[index]))) == 2)
        index+=1
    # print("okidk ", np.average(np.isfinite(outlierValues)))
    # assert(np.average(np.isfinite(outlierValues)) > 0.4)
    return outlierValues

def encode(outlierobjects, buckets, minval=0, maxval=0):
    numberOfObjects = len(outlierobjects)
    X = np.full((numberOfObjects, buckets), np.nan)
    raw = np.concatenate([oo.flatten() for oo in outlierobjects])
    filter = np.isfinite(raw)
    minval = np.min(raw[filter]) if minval == 0 else minval
    maxval = np.max(raw[filter]) if maxval == 0 else maxval
    for index in range(0, numberOfObjects):
        xs = outlierobjects[index]
        filter = np.isfinite(xs)
        hist, edges = np.histogram(xs[filter], bins=buckets, range=(minval, maxval), density=True)
        X[index] = np.array(hist)
    
    return X, minval, maxval, edges

def classify(Xtrain, Ytrain, Xtest, Ytest, n_neighbors, histnormalise, classifier):
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights="uniform") if classifier == 'KNN' else tree.DecisionTreeClassifier()
    clf2 = clf
    scaler = StandardScaler()
    if (histnormalise == True):
        clf = Pipeline([('scaler', scaler), ('classifier', clf)])
    cvs = cross_val_score(clf, Xtrain, Ytrain, cv=5, scoring='roc_auc', n_jobs=-1)
    clf.fit(Xtrain, Ytrain)
    yfit = clf.predict_proba(Xtest)[:,1]
    Xtransformed = scaler.transform(Xtest)
    for i in range(0,10):
        object = Xtrain[i]
        bars = len(object)
        plt.bar(range(0, bars), height=object, width=1, color='tab:green')
        plt.title("Original segment " + str(i%5 + 1) + " Object " + str(i // 5 + 1) + " Label " + str(Ytrain[i]))
        plt.figure()
    for i in range(0,10):
        object = Xtransformed[i]
        bars = len(object)
        plt.bar(range(0, bars), height=object, width=1, color='tab:green')
        plt.title("Transformed segment " + str(i%5 + 1) + " Object " + str(i // 5 + 1) + " Label " + str(Ytrain[i]))
        plt.figure()
    return cvs.mean(), metrics.roc_auc_score(Ytest, yfit), clf2.kneighbors(Xtransformed, return_distance=False) if classifier == 'KNN' else np.array([])

    clf = neighbors.KNeighborsClassifier() if type == 'KNN' else tree.DecisionTreeClassifier()
    clf2 = clf
    scaler = StandardScaler()
    if (histnormalise == True):
        clf = Pipeline([('scaler', scaler), ('classifier', clf)])
    # cvs = cross_val_score(clf, Xtrain, Ytrain, cv=5, scoring='roc_auc', n_jobs=-1)
    clf.fit(Xtrain, Ytrain)
    Yfit = clf.predict(Xtest)
    Yprobs = clf.predict_proba(Xtest)
    Xtransformed = scaler.transform(Xtest)

    return Yfit, Yprobs, clf2.kneighbors(Xtransformed)

In [ ]:
trainobjects, trainobjectinfo, testobjects, testobjectinfo = getTrainObjects(porositythreshold=0.25, objectsplit=3, separate_test=False)

# Show original
plt.imshow(trainobjects[1, 93], vmin=0)
plt.axis('off')
plt.savefig('figures/originaltree.pdf', dpi=300, bbox_inches='tight')
plt.title("Original")
plt.figure()

tmpobject = np.copy(trainobjects[1])
tmpobject[~trainmask] = np.nan
plt.imshow(tmpobject[93], vmin=0)
plt.axis('off')
plt.savefig('figures/nobackground.pdf', dpi=300, bbox_inches='tight')
plt.colorbar()
plt.savefig('figures/nobackgroundcolorbar.pdf', dpi=300, bbox_inches='tight')
plt.title("Background gone")
plt.figure()

# Show outlier
rtn = np.full(trainobjects.shape, np.nan)
rtnscatter = np.full(trainobjects.shape, np.nan)
    # print(rtn.shape)
for index, object in enumerate(trainobjects):
    xs = np.array(object, copy=True, dtype=np.float32)
    xsscatter = np.array(object, copy=True, dtype=np.float32)
    xsscatter[~trainmask] = np.nan
    rtnscatter[index] = xsscatter
    avg = np.mean(xs, where=trainmask)
    stddev = np.std(xs, where=trainmask)
    xs = (xs - avg) / avg
    xs[~trainmask] = np.nan
    rtn[index] = xs

windowSize = 3
objects = rtn

plt.imshow(objects[1, 93])
plt.title("Normalised (for moran)")
plt.figure()

c, z, y, x = objects.shape
outlierValues = np.full((c, z + 1 - windowSize, y, x), np.nan)
for index, object in enumerate(objects):
    # Step 1: calculate neighbourhood
    neighbourkernel = np.ones((3, 3)) / 9
    flatNeighbourhood = np.array([cv2.filter2D(src=layer, ddepth=-1, kernel=neighbourkernel) for layer in object])
    neighbourhoodValues = np.array([
        np.sum(flatNeighbourhood[layerIndex-windowSize:layerIndex], axis=0)/windowSize
        for layerIndex in range(windowSize, z+1)
    ])
    # Step 2: calculate outlier
    offset = windowSize // 2
    endoffset = windowSize - offset - 1

    xs = object[offset:z-endoffset]
    ys = neighbourhoodValues[0:z-windowSize+1]
    filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))

    assert(len(set(filter.flatten())) == 2)    
    line = np.polyfit(ys[filter].flatten(), xs[filter].flatten(), 1)
    p = np.poly1d(line)
    outlierValues[index] = xs - p(ys)
    assert(outlierValues[index].shape == xs.shape == p(ys).shape)
    assert(len(np.unique(outlierValues[index])) > 1)
    assert(len(np.unique(np.isfinite(outlierValues[index]))) == 2)

    if(index == 1):
        plt.scatter(ys[92].flatten(), xs[92].flatten(), c=(xs - p(ys))[92].flatten(), s=10)
        # plt.scatter(ys2[filter2].flatten(), xs2[filter2].flatten(), color='tab:red', s=10, alpha=0.125)
        plt.axline((-0.1, p(-0.1)), (0, p(0)), linewidth=2, color='tab:blue')
        plt.xlabel("Neighbourhood value")
        plt.ylabel("Local value")
        plt.savefig('figures/moran-color-example.pdf', dpi=300, bbox_inches='tight')
        plt.title("Moran scatter plot for points in a high porosity object")
        plt.figure()

assert(np.average(np.isfinite(outlierValues)) > 0.4)
plt.imshow(outlierValues[1, 92])
plt.axis('off')
plt.savefig('figures/moranoutliervalues.pdf', dpi=300, bbox_inches='tight')
plt.colorbar()
plt.savefig('figures/moranoutliercolorbar.pdf', dpi=300, bbox_inches='tight')
plt.title("Moran outlier values")
plt.figure()



objects = rtnscatter

c, z, y, x = objects.shape
outlierValuesScatter = np.full((c, z + 1 - windowSize, y, x), np.nan)
outlierValuesSpatstat = np.full((c, z + 1 - windowSize, y, x), np.nan)
nhood = np.full((c, z + 1 - windowSize, y, x), np.nan)
for index, object in enumerate(objects):
    # Step 1: calculate neighbourhood
    neighbourkernel = np.ones((3, 3)) / 3**2
    flatNeighbourhood = np.array([cv2.filter2D(src=layer, ddepth=-1, kernel=neighbourkernel) for layer in object])
    # funny = np.logical_and(np.isfinite(object), ~np.isfinite(flatNeighbourhood))
    # flatNeighbourhood[funny] = np.nanmean(flatNeighbourhood)
    neighbourhoodValues = np.array([
        np.sum(flatNeighbourhood[layerIndex-windowSize:layerIndex], axis=0)/windowSize
        for layerIndex in range(windowSize, z+1)
    ])
    # Step 2: calculate outlier
    offset = windowSize // 2
    endoffset = windowSize - offset - 1

    xs = object[offset:z-endoffset]
    ys = neighbourhoodValues[0:z-windowSize+1]
    filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))
    funny = np.logical_and(np.isfinite(xs), ~np.isfinite(ys))


    assert(len(set(filter.flatten())) == 2)    
    line = np.polyfit(ys[filter].flatten(), xs[filter].flatten(), 1)
    p = np.poly1d(line)
    outlierValuesScatter[index] = xs - p(ys)
    assert(outlierValuesScatter[index].shape == xs.shape == p(ys).shape)
    assert(len(np.unique(outlierValues[index])) > 1)
    assert(len(np.unique(np.isfinite(outlierValues[index]))) == 2)
    outliers = xs - ys
    avg = np.nanmean(outliers)
    std = np.nanstd(outliers)
    outliers = (outliers - avg) / std
    outlierValuesSpatstat[index] = outliers
    nhood[index] = neighbourhoodValues
    if(index == 1):
        plt.scatter(ys[92].flatten(), xs[92].flatten(), c=(xs - p(ys))[92].flatten(), s=10)
        # plt.scatter(ys2[filter2].flatten(), xs2[filter2].flatten(), color='tab:red', s=10, alpha=0.125)
        plt.axline((10000, p(10000)), (10001, p(10001)), linewidth=2, color='tab:blue')
        plt.xlabel("Neighbourhood value")
        plt.ylabel("Local value")
        plt.savefig('figures/scatter-color-example.pdf', dpi=300, bbox_inches='tight')
        plt.title("Scatter plot for points in a high porosity object")
        plt.figure()

assert(np.average(np.isfinite(outlierValues)) > 0.4)
plt.imshow(outlierValuesScatter[1, 92])
plt.axis('off')
plt.savefig('figures/scatteroutlier.pdf', dpi=300, bbox_inches='tight')
plt.colorbar()
plt.savefig('figures/scatteroutliercolorbar.pdf', dpi=300, bbox_inches='tight')
plt.title("Scatter outlier values")
plt.figure()

plt.imshow(outlierValuesSpatstat[1, 92])
plt.axis('off')
plt.savefig('figures/spatstatoutlier.pdf', dpi=300, bbox_inches='tight')
plt.colorbar()
plt.savefig('figures/spatstatoutliercolorbar.pdf', dpi=300, bbox_inches='tight')
plt.title("Spatstat outlier values")
plt.figure()

